# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Send HTTP request to the target Website

In [6]:
base_url='https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/'

check the response status code

In [7]:
response=requests.get(base_url+'1')
response.status_code

200

### Parse the response content with BeautifulSoap

In [8]:
if response.status_code == 200:
  soup=BeautifulSoup(response.content,'html.parser')

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [9]:
products=soup.select('.product-grid-item')
parsed_products=[]
for product in products:
  title=product.select_one('.wd-entities-title').text.strip()
  prices = product.select('.woocommerce-Price-amount')
  regular_price = prices[0].text.strip() if prices else None
  discount_price = prices[1].text.strip() if len(prices) > 1 else None
  url=product.select_one('.wd-entities-title a')['href']
  button=product.select_one('a.add_to_cart_button')['href']

  product_dictionary={
      'ProductName':title,
      'ProductPrice':regular_price,
      'DiscountedPrice':discount_price,
      'ProductPage':url,
      'AddToCartButton':button
  }
  parsed_products.append(product_dictionary)

Repeat the extraction process for each page of products

In [10]:
def extract_products_as_dict(product):
  title=product.select_one('.wd-entities-title').text.strip()
  prices = product.select('.woocommerce-Price-amount')
  regular_price = prices[0].text.strip() if prices else None
  discount_price = prices[1].text.strip() if len(prices) > 1 else None
  url=product.select_one('.wd-entities-title a')['href']
  button=product.select_one('a.add_to_cart_button')['href']

  product_dictionary={
      'ProductName':title,
      'ProductPrice':regular_price,
      'DiscountedPrice':discount_price,
      'ProductPage':url,
      'AddToCartButton':button
  }
  return product_dictionary

all_products=[]
for i in range(1,14):
  url=base_url+str(i)
  response=requests.get(url)
  soup=BeautifulSoup(response.content,'html.parser')
  products=soup.select('.product-grid-item')
  for product in products:
    result=extract_products_as_dict(product)
    all_products.append(result)


### Create a pandas DataFrame with the scraped products

In [11]:
df=pd.DataFrame(all_products)

Save the dataframe as `.csv`

In [15]:
df.to_csv('products_all_pages.csv', encoding='utf-8-sig')


In [17]:
df.head()

,ProductName,ProductPrice,DiscountedPrice,ProductPage,AddToCartButton
0,Acer A315-23-A7KD,17.590 ден,None,https://clevershop.mk/product/acer-a315-23-a7kd/,?add-to-cart=21494
1,Acer A315-23-R5P2,27.490 ден,None,https://clevershop.mk/product/acer-a315-23-r5p2/,?add-to-cart=21510
2,ACER Aspire 1 A115-22,18.999 ден,15.999 ден,https://clevershop.mk/product/acer-aspire-1-nx...,?add-to-cart=20826
3,Acer Aspire 3 A315-23-R26A,29.990 ден,None,https://clevershop.mk/product/acer-aspire-3-a3...,?add-to-cart=21516
4,Acer Aspire 3 A315-58-33WK,24.490 ден,None,https://clevershop.mk/product/21498/,?add-to-cart=21498
